In [1]:
# from google.colab import drive
# drive.mount('/content/GoogleDrive')

In [ ]:
!pip install split-folders

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf

from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, zero_one_loss, classification_report
from sklearn.model_selection import train_test_split
from imutils import paths

import matplotlib.pyplot as plt
import seaborn as sb
import splitfolders
import cv2
import random

In [4]:
inputFolder = 'C:\\Users\\user\\Desktop\\Graduation Project\\DataSets\\Brain Cancer'
outputFolder = 'C:\\Users\\user\\Desktop\\Graduation Project\\image_dataset_(Brain)'

# inputFolder = '/content/GoogleDrive/MyDrive/Colab Notebooks/Neural Networks/Graduation Project/Brain Cancer'
# outputFolder = '/content/GoogleDrive/MyDrive/Colab Notebooks/Neural Networks/Graduation Project/image_dataset_(Brain)'

In [ ]:
splitfolders.ratio(inputFolder, outputFolder, seed = 44, ratio = (0.8, 0.0, 0.2))

In [6]:
imgPaths = list(paths.list_images(outputFolder))
print(imgPaths[0].split('\\'))

random.shuffle(imgPaths)

classes = [] # classes = ['brain_menin', 'brain_glioma', 'brain_pituitary', 'no_tumor']
for imgPath in imgPaths:
  className = imgPath.split('\\')[-2]

  if className not in classes:
    classes.append(className)

print(classes)

X_train = []
y_train = []

X_test = []
y_test = []

IMGSIZE = 227
img_names = []

for imgPath in imgPaths:
  trainOrTest = imgPath.split('\\')[-3]
  className = imgPath.split('\\')[-2]

  img = cv2.imread(imgPath) #cv2.IMREAD_GRAYSCALE
  img = cv2.resize(img, (IMGSIZE,IMGSIZE))

  if trainOrTest == 'train':
    X_train.append(img)
    y_train.append(classes.index(className))
    img_names.append(imgPath.split('\\')[-1])

  elif trainOrTest == 'test':
    X_test.append(img)
    y_test.append(classes.index(className))

print(len(X_train), len(y_train))
print(len(X_test), len(y_test))

['C:', 'Users', 'user', 'Desktop', 'Graduation Project', 'image_dataset_(Brain)', 'test', 'brain_glioma', 'brain_glioma_0001.jpg']
['C:', 'Users', 'user', 'Desktop', 'Graduation Project', 'image_dataset_(Brain)', 'test', 'brain_glioma', 'brain_glioma_0001.jpg']
['no_tumor', 'brain_menin', 'brain_glioma', 'brain_pituitary']
['no_tumor', 'brain_menin', 'brain_glioma', 'brain_pituitary']
16000 16000
4000 4000
16000 16000
4000 4000


In [ ]:
df = pd.DataFrame({
    'File Name' : img_names,
    'Category' : y_train #y
})

# print(classes.index('brain_menin'))
# print(classes.index('brain_glioma'))
# print(classes.index('brain_pituitary'))
# print(classes.index('no_tumor'))

print(classes)
df.head()

In [ ]:
sb.countplot(x = y_train)

In [ ]:
sb.countplot(x = y_test)

In [11]:
X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [12]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(16000, 227, 227, 3) (16000,)
(4000, 227, 227, 3) (4000,)
(16000, 227, 227, 3) (16000,)
(4000, 227, 227, 3) (4000,)


In [13]:
model = Sequential()

model.add(Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(IMGSIZE, IMGSIZE, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

model.add(Conv2D(filters=256, kernel_size=(5, 5), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

model.add(Conv2D(filters=384, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(filters=384, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(4, activation='softmax'))
          
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
results = model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 20, batch_size = 32 , callbacks = [early_stopping])
# val_loss, val_acc = model.evaluate(X_test, y_test)

y_pred = model.predict(X_test)

model.summary()

Epoch 1/20
Epoch 1/20
500/500 [==============================] - 2041s 4s/step - loss: 1.6001 - accuracy: 0.2654 - val_loss: 1.3871 - val_accuracy: 0.2500
Epoch 2/20
Epoch 2/20
500/500 [==============================] - 2025s 4s/step - loss: 1.3874 - accuracy: 0.2407 - val_loss: 1.3864 - val_accuracy: 0.2500
Epoch 3/20
Epoch 3/20
500/500 [==============================] - 2021s 4s/step - loss: 1.3869 - accuracy: 0.2466 - val_loss: 1.3864 - val_accuracy: 0.2500
Epoch 4/20
Epoch 4/20
500/500 [==============================] - 2027s 4s/step - loss: 1.3867 - accuracy: 0.2555 - val_loss: 1.3865 - val_accuracy: 0.2500
Epoch 5/20
Epoch 5/20
500/500 [==============================] - 2023s 4s/step - loss: 1.3867 - accuracy: 0.2472 - val_loss: 1.3866 - val_accuracy: 0.2500
Epoch 6/20
Epoch 6/20
500/500 [==============================] - 2019s 4s/step - loss: 1.3866 - accuracy: 0.2499 - val_loss: 1.3865 - val_accuracy: 0.2500
Epoch 7/20
Epoch 7/20
125/125 [==============================] - 96s 7

In [ ]:
model.save('C:\\Users\\user\\Desktop\\Graduation Project\\model_image_(Brain)_AlexNet.h5')

# model.save('/content/GoogleDrive/MyDrive/Colab Notebooks/Neural Networks/Graduation Project/model_image_(Brain).h5')
# loaded_model = load_model('/content/GoogleDrive/MyDrive/Colab Notebooks/Neural Networks/Graduation Project/model_image_(Brain).h5')

In [ ]:
plt.plot(results.history['accuracy'], label = 'Training Accuracy')
plt.plot(results.history['val_accuracy'], label = 'Testing Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(results.history['loss'], label = 'Training Loss')
plt.plot(results.history['val_loss'], label = 'Testing Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
print(classes)
print(y_test[:25], '\n')
print(y_pred[:25], '\n')

predict = []
for i in range(len(y_pred)):
  predict.append(np.argmax(y_pred[i]))

print(predict[:25], '\n')

cm = confusion_matrix(y_test, predict)
print(cm, '\n')

zol = zero_one_loss(y_test, predict, normalize = False)
print(zol, '\n')

cr = classification_report(y_test, predict)
print(cr)

In [ ]:
plt.figure(figsize = (5,5))
ConfusionMatrixDisplay(cm).plot()
plt.show()

In [ ]:
sb.countplot(x = predict)

In [ ]:
plot_model(model, show_shapes = True)

In [ ]:
test_img_path = 'C:\\Users\\user\\Desktop\\Graduation Project\\image_dataset_(Brain)\\test\brain_glioma\\brain_glioma_0077.jpg'
# test_img_path = '/content/GoogleDrive/MyDrive/Colab Notebooks/Neural Networks/Graduation Project/image_dataset_(Brain)/test/brain_glioma/brain_glioma_4736.jpg'
test_img = cv2.imread(test_img_path)
test_img = cv2.resize(test_img , (IMGSIZE,IMGSIZE))

plt.imshow(test_img)

test_img = test_img[tf.newaxis, ...]
test_predict = model.predict(test_img)

str1 = test_img_path.split('\\')[-1].split('_')[0]
str2 = test_img_path.split('\\')[-1].split('_')[1]
str3 = classes[np.argmax(test_predict)].split('_')[0] + ' ' + classes[np.argmax(test_predict)].split('_')[1]

plt.title('Actual: ' + str1 + ' ' + str2 + '\n Predicted: ' + str3)

print(test_predict)
print(classes)
plt.show()